In [9]:
import networkx as nx
import numpy as np
import pandas as pd

In [10]:
def genKGBERTSageDataFromGraph(G=nx.MultiDiGraph(), datasetUsage='Train'):

    all_nodes = list(nx.nodes(G))
    all_edges = list(nx.edges(G))

    id2node = dict(enumerate(all_nodes,1))
    node2id = {node: i for i, node in enumerate(all_nodes,1)}
    id2edge = dict(enumerate(all_edges,1))
    edge2id = {edge: i for i, edge in enumerate(all_edges,1)}
    np.save('./KGBertSageData/ID2NODE_{}.npy'.format(datasetUsage), id2node,allow_pickle=True)
    np.save( './KGBertSageData/NODE2ID_{}.npy'.format(datasetUsage), node2id,allow_pickle=True)
    np.save( './KGBertSageData/ID2EDGE_{}.npy'.format(datasetUsage),id2edge,allow_pickle=True)
    np.save( './KGBertSageData/EDGE2ID_{}.npy'.format(datasetUsage),edge2id,allow_pickle=True)

    edgeid2relation = {}
    for edgeID in list(id2edge.keys()):
        relation = G.get_edge_data(id2edge[edgeID][0],id2edge[edgeID][1])
        edgeid2relation[edgeID] = relation

    np.save( './KGBertSageData/EDGEID2Relation_{}.npy'.format(datasetUsage),edgeid2relation,allow_pickle=True)

    node_id = list(np.arange(len(all_nodes))+1)
    node = all_nodes

    neighbor_edge = []

    for each_node in all_nodes:
        its_edges = []
        its_tail = list(G.successors(each_node))
        its_head = list(G.predecessors(each_node))
        for each_its_tail in its_tail:
            its_edges.append(edge2id[(each_node, each_its_tail)])
        for each_its_head in its_head:
            its_edges.append(edge2id[(each_its_head, each_node)])
        neighbor_edge.append(its_edges)
    
    sample = {
        'Node_ID': node_id,
        'Node': node,
        'Neighbor_Edge': neighbor_edge,
    }

    df = pd.DataFrame(sample)
    df.to_csv('./KGBertSageData/KGBertSageData_{}.tsv'.format(datasetUsage), sep='\t',index=False)

In [11]:
G = nx.read_gml('./GenerateATOMICGraph/ATOMIC2020_14Relations_dev.gml')

In [12]:
genKGBERTSageDataFromGraph(G,'Dev')